In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as fn
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay
%matplotlib inline

In [ ]:
# Run the dataset preparation notebook
%run Dataset_preparation.ipynb
# outputs training and test sets (X and y)

In [ ]:
class Model(nn.Module): #inherets torch.nn.Module
    # input 8 features
    # hidden layer 1 
    # hidden layer 2
    # output layer (star, galaxy, quasar)
    def __init__(self,in_features=8, h1=32, h2=16, out_features=3):
        super().__init__()
        self.fc1 = nn.Linear(in_features, h1)
        self.fc2 = nn.Linear(h1, h2)
        self.out = nn.Linear(h2, out_features)

    def forward(self, x):
        x = fn.tanh(self.fc1(x))
        x = fn.tanh(self.fc2(x))
        x = self.out(x)

        return x

In [ ]:
# torch.manual_seed(17) # commented out after testing is done
model = Model()

In [ ]:
# Loss
criterion = nn.CrossEntropyLoss()
# optimizer => Adam
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

# Training

In [ ]:
epochs = 1000
losses = []
accurate = 0
last_accuracy = 0
for i in range(epochs):
    # forward
    y_pred = model.forward(X_train)
    
    # loss
    loss = criterion(y_pred, y_train)
    losses.append(loss.detach().numpy())

    # track recent accuracy
    if (y_pred[i].argmax()== y_train[i]):
        accurate += 1

    if i % 100 == 0 :
        last_accuracy = accurate
        print(f' Epoch: {i} and loss: {loss} \t recent accuracy:{accurate/10}')
        accurate = 0

    # back propagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
# model evaluation
with torch.no_grad(): # disable learning
    y_eval = model.forward(X_test)
    loss = criterion(y_eval, y_test)
print(f'Test loss: {loss}')


In [ ]:
# accuracy
runs = 0
accurate = 0
y_pred=[]
with torch.no_grad(): # disable learning
    for i, data in enumerate(X_test):
        runs = i
        y_eval = model.forward(data)
        y_pred.append(y_eval.argmax().item())

        if (y_eval.argmax() == y_test[i]):
            accurate += 1
print(f' accuracy: {accurate/runs}')

In [ ]:
# plot model performance
print(f' accuracy: {accurate/runs}')
ConfusionMatrixDisplay.from_predictions(y_test, y_pred)
plt.show()